# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential

import numpy as np
import PIL
import PIL.Image

# Data Loading

In [19]:
#https://keras.io/api/data_loading/

BATCH_SIZE = 32
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 256
IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

train_ds = keras.utils.image_dataset_from_directory(
    directory='coin_dataset/coins_images/currency_recognition',
    labels='inferred',
    label_mode='int',
    validation_split=0.05,
    subset="training",
    seed=212,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True)

val_ds = keras.utils.image_dataset_from_directory(
    directory='coin_dataset/coins_images/currency_recognition',
    labels='inferred',
    label_mode='int',
    validation_split=0.05,
    subset="validation",
    seed=212,
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True)

class_names = train_ds.class_names
print(class_names)


Found 1229 files belonging to 6 classes.
Using 1168 files for training.
Found 1229 files belonging to 6 classes.
Using 61 files for validation.
['china_coins', 'euro_coins', 'indian_coins', 'peso_coins', 'us_coins', 'yen_coins']


In [ ]:
#https://www.tensorflow.org/tutorials/load_data/images

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


# Model

In [20]:
#https://www.tensorflow.org/tutorials/images/classification#visualize_training_results

num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
    layers.Conv2D(8, (5, 5), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
    layers.Dropout(0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.Dropout(0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes)
])

In [21]:
model.compile(
  optimizer='nadam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 128, 256, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 128, 256, 8)       608       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 64, 128, 8)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 64, 128, 16)       1168      
                                                                 
 dropout_9 (Dropout)         (None, 64, 128, 16)       0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 32, 64, 16)       0         
 g2D)                                                 

In [18]:
#https://machinelearningmastery.com/image-augmentation-deep-learning-keras/
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(train_ds.as_numpy())

x_batch, y_batch = datagen.flow(train_ds, train_ds, batch_size=BATCH_SIZE)

AttributeError: 'BatchDataset' object has no attribute 'as_numpy'

In [22]:
epochs=15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
37/37 [==============================] - 7s 153ms/step - loss: 1.8487 - accuracy: 0.2029 - val_loss: 1.8153 - val_accuracy: 0.0984
Epoch 2/15
37/37 [==============================] - 6s 152ms/step - loss: 1.7330 - accuracy: 0.2654 - val_loss: 1.7974 - val_accuracy: 0.1639
Epoch 3/15
37/37 [==============================] - 6s 161ms/step - loss: 1.6472 - accuracy: 0.3373 - val_loss: 1.8073 - val_accuracy: 0.2623
Epoch 4/15
37/37 [==============================] - 6s 151ms/step - loss: 1.4601 - accuracy: 0.4315 - val_loss: 1.6495 - val_accuracy: 0.3443
Epoch 5/15
37/37 [==============================] - 6s 157ms/step - loss: 1.2593 - accuracy: 0.5420 - val_loss: 1.5578 - val_accuracy: 0.3443
Epoch 6/15
37/37 [==============================] - 6s 154ms/step - loss: 0.9134 - accuracy: 0.6695 - val_loss: 1.3859 - val_accuracy: 0.4590
Epoch 7/15
37/37 [==============================] - 6s 152ms/step - loss: 0.7205 - accuracy: 0.7423 - val_loss: 1.4794 - val_accuracy: 0.5246
Epoch 

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
